## Praca z tekstami KERAS

### Ładowanie danych

Zbiory: 25 000 recenzji filmów z IMDB, oznaczonych sentymentem (pozytywne / negatywne). Opinie były wstępnie przetworzone, każda jest zakodowana jako sekwencja indeksów słów (liczby całkowite). Dla wygody słowa są indeksowane z uwzględnieniem częstotliwości wystąpień, dzięki czemu na przykład liczba całkowita "3" koduje trzecie najczęściej występujące słowo w danych. Pozwala to na szybkie operacje filtrowania, takie jak: "uwzględnij tylko 10.000 najczęściej używanych słów, ale wyeliminuj 20 najbardziej popularnych słów".

Zgodnie z konwencją "0" nie oznacza konkretnego słowa, ale używa się do kodowania nieznanego słowa.

In [15]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb
import numpy as np


<module 'numpy.version' from '/opt/anaconda3/lib/python3.6/site-packages/numpy/version.py'>


Parametry:

In [16]:
print(np.version)
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

<module 'numpy.version' from '/opt/anaconda3/lib/python3.6/site-packages/numpy/version.py'>


In [14]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(path="imdb.npz", num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

ValueError: Object arrays cannot be loaded when allow_pickle=False

Standaryzacja długości sekwencji (znalezienie najdłuższej, wypełnienie zerami pozostałych w taki sposób, aby wszystkie były jednakowej długości)

In [4]:
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

('x_train shape:', (25000, 400))
('x_test shape:', (25000, 400))


### Budowanie modelu

Składa się z warstwy word embeddingów, następnie dropout, konwolucja/splot 1d, max pooling, dense, dropout i wyjście

In [5]:
model = Sequential()
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

Uruchamianie modelu:

In [6]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 350s - loss: 0.4112 - acc: 0.8032 - val_loss: 0.3055 - val_acc: 0.8709

ĆWICZENIA:

1. Przepisz model sieci na funkcjonalne API

2. Zmień typ sieci na rekurencyjną (RNN), użyj komórki typu LSTM, zostawiając z pokazanego modelu tylko ostatnią warstwę Dense
https://keras.io/layers/recurrent/

3. Uruchom eksperyment z prostą siecią RNN (SimpleRNN), czy i na ile wyniki zmieniają się?

4. Wypróbuj użycie gotowych embeddingów glove   https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html    https://github.com/fchollet/keras/blob/master/examples/pretrained_word_embeddings.py   - czy i na ile wyniki zmieniają się?